In [97]:
!python -m pip install -q -U antlr4-python3-runtime

In [98]:
!curl -O https://www.antlr.org/download/antlr-4.13.2-complete.jar &> /dev/null

In [99]:
!javac -version

javac 11.0.24


In [100]:
%%file ImageProcessing.g4
grammar ImageProcessing;

program : statement+ ;

statement
	: loadImage
	| saveImage
	| applyFilter
	| transform
	;

// Cargar imagen
loadImage : 'load' STRING 'as' ID ';' ;

// Guardar imagen
saveImage : 'save' ID 'to' STRING ';' ;

// Aplicar filtros
applyFilter : ID '.' 'filter' '(' filterType ')' ';' ;


filterType
	: 'blur'
	| 'sharpen'
	| 'grayscale'
	| 'edgeDetection'
	;

// Transformaciones
transform : ID '.' transformation '(' transformationValue ')' ';' ;

transformation
	: 'rotate'
	| 'resize'
	| 'crop'
	;

transformationValue
	: NUMBER // Para valores de rotación, tamaño, etc.
	;

// Tokens
STRING : '"' .*? '"' ; // Para nombres de archivo
ID : [a-zA-Z]+ ; // Identificadores
NUMBER : [0-9]+('.'[0-9]+)?;
WS : [ \t\r\n]+ -> skip ; // Ignorar espacios en blanco

Overwriting ImageProcessing.g4


In [101]:
!java -cp antlr-4.13.2-complete.jar:. org.antlr.v4.Tool -Dlanguage=Python3 ImageProcessing.g4

In [102]:
%%file ejemplo.ImageProcessing
load "cocodrile.jpg" as img;
img.filter(grayscale);
save img to "finalImage.jpg";

Overwriting ejemplo.ImageProcessing


In [103]:
from ImageProcessingLexer import ImageProcessingLexer
from ImageProcessingParser import ImageProcessingParser
from ImageProcessingListener import ImageProcessingListener

from antlr4.CommonTokenStream import CommonTokenStream
from antlr4 import FileStream

In [104]:
inputFile = FileStream("ejemplo.ImageProcessing")
lexer = ImageProcessingLexer(inputFile)
tokens = CommonTokenStream(lexer)
parser = ImageProcessingParser(tokens)
tree = parser.program()
print(tree.toStringTree(recog=parser))

(program (statement (loadImage load "cocodrile.jpg" as img ;)) (statement (applyFilter img . filter ( (filterType grayscale) ) ;)) (statement (saveImage save img to "finalImage.jpg" ;)))


In [105]:
import cv2

images = {}

class ImageProcessingDriver(ImageProcessingListener):

    def enterLoadImage(self, ctx):
        file_path = ctx.STRING().getText().strip('"')
        image_name = ctx.ID().getText()
        image = cv2.imread(file_path)
        if image is not None:
            images[image_name] = image
            print(f"Imagen '{file_path}' cargada como '{image_name}'.")
        else:
            print(f"Error: no se pudo cargar la imagen '{file_path}'.")

    def enterSaveImage(self, ctx):
        image_name = ctx.ID().getText()
        file_path = ctx.STRING().getText().strip('"')
        if image_name in images:
            cv2.imwrite(file_path, images[image_name])
            print(f"Imagen '{image_name}' guardada en '{file_path}'.")
        else:
            print(f"Error: la imagen '{image_name}' no existe.")

    def enterApplyFilter(self, ctx):
        image_name = ctx.ID().getText()
        filter_type = ctx.filterType().getText()
        if image_name in images:
            if filter_type == 'blur':
                images[image_name] = cv2.GaussianBlur(images[image_name], (5, 5), 0)
                print(f"Filtro 'blur' aplicado a la imagen '{image_name}'.")
            elif filter_type == 'grayscale':
                images[image_name] = cv2.cvtColor(images[image_name], cv2.COLOR_BGR2GRAY)
                print(f"Filtro 'grayscale' aplicado a la imagen '{image_name}'.")
            elif filter_type == 'edgeDetection':
                images[image_name] = cv2.Canny(images[image_name], 100, 200)
                print(f"Filtro 'edgeDetection' aplicado a la imagen '{image_name}'.")
        else:
            print(f"Error: la imagen '{image_name}' no existe.")

    def enterTransform(self, ctx):
        image_name = ctx.ID().getText()
        transformation = ctx.transformation().getText()
        value = int(ctx.transformationValue().getText())
        if image_name in images:
            if transformation == 'rotate':
                (h, w) = images[image_name].shape[:2]
                center = (w // 2, h // 2)
                M = cv2.getRotationMatrix2D(center, value, 1.0)
                images[image_name] = cv2.warpAffine(images[image_name], M, (w, h))
                print(f"Imagen '{image_name}' rotada {value} grados.")
            elif transformation == 'resize':
                images[image_name] = cv2.resize(images[image_name], (value, value))
                print(f"Imagen '{image_name}' redimensionada a {value}x{value} píxeles.")
        else:
            print(f"Error: la imagen '{image_name}' no existe.")

In [106]:
from antlr4 import ParseTreeWalker

input = FileStream("ejemplo.ImageProcessing")
lexer = ImageProcessingLexer(input)
tokens = CommonTokenStream(lexer)
parser = ImageProcessingParser(tokens)
tree = parser.program()
walker = ParseTreeWalker()
walker.walk(ImageProcessingDriver(), tree)

Error: no se pudo cargar la imagen 'cocodrile.jpg'.
Error: la imagen 'img' no existe.
Error: la imagen 'img' no existe.
